## Plug the File name here

In [8]:
#################################
#**  Change file name below  **

testfile_name = "data/testDatasetExample.xls"

#*********************************




### Import all necessary packages

In [9]:
import tensorflow as tf
from tensorflow import keras

import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn


from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC, SVR
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import f_classif

from scipy import stats as st
from sklearn.preprocessing import PowerTransformer

import pickle

### Reading the data

In [10]:
test_df = pd.read_excel(testfile_name)

### Loading all the model objects

In [11]:
# load the imputer from disk
filename = 'saved_model/int_cols_imputer.sav'
sic = pickle.load(open(filename, 'rb'))

filename = 'saved_model/float_cols_imputer.sav'
sif = pickle.load(open(filename, 'rb'))

# load the transformer from disk
filename = 'saved_model/trnsformer.sav'
transformer = pickle.load(open(filename, 'rb'))


# load the scaler from disk
filename = 'saved_model/scaler.sav'
scaler = pickle.load(open(filename, 'rb'))

# Continuous columns list
float_cols = pickle.load(open('saved_model/continuous_features.pkl', 'rb'))['continuous_features']

# Categorical columns
cat_cols = pickle.load(open('saved_model/categorical_features.pkl', 'rb'))['categorical_features']

## Loadin the model
model = tf.keras.models.load_model('saved_model/best_model_aucc82')


# Preprocessing data

**Making the data frame from the  Columns used in the training**

In [12]:
df_cats = test_df[cat_cols]
df_floats = test_df[float_cols]

### Filling missing values

In [14]:
df_cats = pd.DataFrame(sic.transform(df_cats),columns=df_cats.columns)
df_floats = pd.DataFrame(sif.transform(df_floats),columns=df_floats.columns)

### Making features normal

In [15]:
df_floats = pd.DataFrame(transformer.transform(df_floats),columns=df_floats.columns)

### Merging features

In [16]:
X = pd.concat([df_cats.reset_index(drop=True),df_floats.reset_index(drop=True)],axis=1)

### Scalling the data

In [17]:
X = pd.DataFrame(scaler.transform(X),columns=X.columns)

### Predict the values

In [18]:
predictions = model.predict(X)

1/1 [==============================] - 1s 860ms/step


### Fomatting the predictions

In [19]:
preds_df = pd.DataFrame(zip(test_df["ID"].to_list(),
                            list(np.squeeze(predictions))),
                            columns=(["ID","PCR"]))

### Writing result to csv

In [20]:
preds_df.to_csv('pcr_predictions.csv',index=False)